In [67]:
from stepPlay import * 
import numpy as np
import copy

In [68]:

def prn_obj(obj):
    print('\n'.join(['%s:%s' % item for item in obj.__dict__.items()]))

In [69]:
def softmax(x):
    probs = np.exp(x - np.max(x))
    probs /= np.sum(probs)
    return probs

In [70]:
n = 5
width, height = 8, 8
model_file =  'best_policy_8_8_5.model'

board = Board(width=width, height=height, n_in_row=n)
game = Game(board)

# ############### human VS AI ###################
# load the trained policy_value_net in either Theano/Lasagne, PyTorch or TensorFlow

# best_policy = PolicyValueNet(width, height, model_file = model_file)
# mcts_player = MCTSPlayer(best_policy.policy_value_fn, c_puct=5, n_playout=400)

# load the provided model (trained in Theano/Lasagne) into a MCTS player written in pure numpy
try:
    policy_param = pickle.load(open(model_file, 'rb'))
except:
    policy_param = pickle.load(open(model_file, 'rb'),
                               encoding='bytes')  # To support python3
#得到策略                               
best_policy = PolicyValueNetNumpy(width, height, policy_param)
#得到策略函数
mcts_player1 = MCTSPlayer(best_policy.policy_value_fn,
                         c_puct=5,
                         n_playout=400)  # set larger n_playout for better performance

mcts_player2 = MCTSPlayer(best_policy.policy_value_fn,
                         c_puct=5,
                         n_playout=400)  

# uncomment the following line to play with pure MCTS (it's much weaker even with a larger n_playout)
# mcts_player = MCTS_Pure(c_puct=5, n_playout=1000)

# human player, input your move in the format: 2,3
human = Human()

player1 =  mcts_player1
player2 =  mcts_player2
start_player=0 
is_shown=1

game.board.init_board(start_player)
p1, p2 = game.board.players
player1.set_player_ind(p1)
player2.set_player_ind(p2)
players = {p1: player1, p2: player2}

In [71]:
##计算概率
current_player = game.board.get_current_player()
player_in_turn = players[current_player]
board=game.board
temp=1e-3
return_prob=0

sensible_moves = board.availables
move_probs = np.zeros(board.width*board.height)
acts, probs = player_in_turn.mcts.get_move_probs(board, temp)
print(acts)
print(probs)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63)
[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 5.91871068e-107
 1.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 1.53059365e-139
 2.92327048e-039 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.000

In [72]:
#计算概率
current_player = game.board.get_current_player()
player_in_turn = players[current_player]
board=game.board
temp=1e-3
return_prob=0

sensible_moves = board.availables
print("---------sensible_moves----------")
print(sensible_moves)
move_probs = np.zeros(board.width*board.height)

state=board
temp=1e-3


for n in range(player_in_turn.mcts._n_playout):
    state_copy = copy.deepcopy(state)
    player_in_turn.mcts._playout(state_copy)

# calc the move probabilities based on visit counts at the root node
act_visits = [(act, node._n_visits)
              for act, node in player_in_turn.mcts._root._children.items()]
acts, visits = zip(*act_visits)
act_probs = softmax(1.0/temp * np.log(np.array(visits) + 1e-10))
print("---------acts----------")
print(acts)
print(visits)
print(act_probs)


move_visits = np.zeros(board.width*board.height)
move_visits[list(acts)] = visits

print(move_visits)

---------sensible_moves----------
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]
---------acts----------
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 4, 2, 12, 0, 0, 0, 0, 2, 157, 214, 4, 0, 0, 0, 0, 3, 164, 206, 4, 0, 0, 0, 0, 7, 3, 3, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.

In [73]:
print( player_in_turn.mcts._root._children.items())

dict_items([(0, <mcts_alphaZero.TreeNode object at 0x0000026F59186EB8>), (1, <mcts_alphaZero.TreeNode object at 0x0000026F59186EF0>), (2, <mcts_alphaZero.TreeNode object at 0x0000026F59186F28>), (3, <mcts_alphaZero.TreeNode object at 0x0000026F59186F60>), (4, <mcts_alphaZero.TreeNode object at 0x0000026F59186F98>), (5, <mcts_alphaZero.TreeNode object at 0x0000026F59186FD0>), (6, <mcts_alphaZero.TreeNode object at 0x0000026F59188048>), (7, <mcts_alphaZero.TreeNode object at 0x0000026F59188080>), (8, <mcts_alphaZero.TreeNode object at 0x0000026F591880B8>), (9, <mcts_alphaZero.TreeNode object at 0x0000026F591880F0>), (10, <mcts_alphaZero.TreeNode object at 0x0000026F59188128>), (11, <mcts_alphaZero.TreeNode object at 0x0000026F59188160>), (12, <mcts_alphaZero.TreeNode object at 0x0000026F59188198>), (13, <mcts_alphaZero.TreeNode object at 0x0000026F591881D0>), (14, <mcts_alphaZero.TreeNode object at 0x0000026F59188208>), (15, <mcts_alphaZero.TreeNode object at 0x0000026F59188240>), (16, <

In [74]:
def printNode(node,level=0,act=0):    
    if(node._n_visits<2):
        return
    str = '	'*level
    print("%s, act=%d,_n_visits=%d ,_Q=%f ,_u=%f ,_P=%f " %(str,act,node._n_visits,node._Q,node._u,node._P))
          
    for act,child in node._children.items():
          printNode(child,level+1,act)

prn_obj(game.board)          
printNode(player_in_turn.mcts._root)          

states:{}
n_in_row:5
width:8
last_move:-1
current_player:1
players:[1, 2]
availables:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]
height:8
, act=0,_n_visits=800 ,_Q=-0.455971 ,_u=0.000000 ,_P=1.000000 
	, act=18,_n_visits=8 ,_Q=-0.185447 ,_u=0.781758 ,_P=0.049782 
		, act=27,_n_visits=3 ,_Q=-0.035337 ,_u=1.092187 ,_P=0.330246 
		, act=36,_n_visits=2 ,_Q=0.365520 ,_u=0.683974 ,_P=0.155111 
	, act=19,_n_visits=4 ,_Q=0.219712 ,_u=0.339472 ,_P=0.012010 
		, act=28,_n_visits=3 ,_Q=-0.214190 ,_u=1.373586 ,_P=0.475824 
			, act=27,_n_visits=2 ,_Q=0.232468 ,_u=1.274235 ,_P=0.360408 
	, act=20,_n_visits=2 ,_Q=0.080553 ,_u=0.501082 ,_P=0.010636 
	, act=21,_n_visits=12 ,_Q=0.058928 ,_u=0.458884 ,_P=0.042209 
		, act=27,_n_visits=2 ,_Q=-0.063028 ,_u=0.487490 ,_P=0.088190 
		, act=28,_n_visits=5 ,_Q=

In [75]:
#执行一个步骤
current_player = game.board.get_current_player()
player_in_turn = players[current_player]
#get_action 得到当前动作
move = player_in_turn.get_action(game.board)
print("current_player=%d , move=%d " %(current_player,move))
game.board.do_move(move)
prn_obj(game.board)

current_player=1 , move=35 
states:{35: 1}
n_in_row:5
width:8
last_move:35
current_player:2
players:[1, 2]
availables:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]
height:8


In [76]:
print(game.board.current_state())

[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]]


In [77]:
game.graphic(game.board, player1.player, player2.player)

Player 1 with X
Player 2 with O

       0       1       2       3       4       5       6       7

   7   _       _       _       _       _       _       _       _    


   6   _       _       _       _       _       _       _       _    


   5   _       _       _       _       _       _       _       _    


   4   _       _       _       X       _       _       _       _    


   3   _       _       _       _       _       _       _       _    


   2   _       _       _       _       _       _       _       _    


   1   _       _       _       _       _       _       _       _    


   0   _       _       _       _       _       _       _       _    


